In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
from time import sleep
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import json


In [2]:
# Đường dẫn lưu bookmark
BOOKMARK_PATH = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/bookmark.csv"

OUTPUT_DIR = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files"

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

def load_bookmark(file_path):
    """Load danh sách liên kết đã cào từ file."""
    if os.path.exists(file_path):
        return set(pd.read_csv(file_path)['Links'])
    return set()

def save_bookmark(file_path, links):
    """Lưu danh sách liên kết đã cào vào file (thêm mới, không ghi đè)."""
    if os.path.exists(file_path):
        existing_links = load_bookmark(file_path)
        links = links.union(existing_links)  # Gộp liên kết cũ và mới
    df = pd.DataFrame({'Links': list(links)})
    df.to_csv(file_path, index=False)

def save_category_links(category_idx, category_links):
    """Lưu các liên kết của danh mục vào file Excel riêng."""
    output_file = os.path.join(OUTPUT_DIR, f"category_{category_idx}.xlsx")
    df = pd.DataFrame({'Links': list(category_links)})
    df.to_excel(output_file, index=False)
    print(f"Liên kết danh mục {category_idx} đã được lưu vào {output_file}.")

def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(2)  # Đợi 2 giây sau khi cuộn xong

def collect_product_links(driver, product_links, max_links=5000):
    """Thu thập liên kết sản phẩm từ trang hiện tại."""
    product_elements = driver.find_elements(By.CSS_SELECTOR, ".styles__ProductItemContainerStyled-sc-bszvl7-0 a.style__ProductLink-sc-139nb47-2")
    for elem in product_elements:
        if len(product_links) >= max_links:
            print(f"Collected {len(product_links)}/{max_links}. Đã đạt giới hạn sản phẩm.")
            break
        link = elem.get_attribute("href")
        if link and link not in product_links:
            product_links.add(link)
            print(f"Collected {len(product_links)}/{max_links}")

def crawl_category_links(driver, category_links, max_links_per_category=5000):
    """Duyệt qua các danh mục và thu thập liên kết sản phẩm."""

    for idx, href in enumerate(category_links, start=1):
        print(f"\nĐang truy cập danh mục {idx}: {href}")
        driver.get(href)
        time.sleep(random.uniform(2, 5))

        category_product_links = set()

        for _ in range(10):  # Nhấn nút "Xem thêm" tối đa 10 lần
            if len(category_product_links) >= max_links_per_category:
                print(f"Đã thu thập đủ {max_links_per_category} liên kết từ danh mục {idx}.")
                break
            try:
                # Thu thập liên kết sản phẩm
                collect_product_links(driver, category_product_links, max_links=max_links_per_category)

                # Nhấn nút "Xem thêm"
                button = driver.find_element(By.CSS_SELECTOR, '[data-view-id="category_infinity_view.more"]')
                ActionChains(driver).move_to_element(button).click(button).perform()
                time.sleep(random.uniform(2, 3))

                # Cuộn trang để tải thêm sản phẩm
                scroll_page(driver, scroll_count=5, pause=random.uniform(1, 1.5))
            except NoSuchElementException:
                print("Nút 'Xem thêm' không còn hoặc đã tải hết sản phẩm.")
                break

        print(f"Danh mục {idx} hoàn thành. Thu thập được {len(category_product_links)} liên kết.")

        # Lưu liên kết của danh mục vào file Excel riêng
        save_category_links(idx, category_product_links)

def main():
    # Tải bookmark đã cào
    crawled_links = load_bookmark(BOOKMARK_PATH)

    # Khởi tạo WebDriver
    driver = webdriver.Chrome()
    driver.get("https://tiki.vn")
    driver.implicitly_wait(5)

    # Tìm tất cả các thẻ <a> bên trong class "styles__StyledListItem-sc-w7gnxl-0"
    category_elements = driver.find_elements(By.CSS_SELECTOR, ".styles__StyledListItem-sc-w7gnxl-0 a")
    category_links = [link.get_attribute("href") for link in category_elements if link.get_attribute("href") not in crawled_links]
    category_links = category_links[1:26]  # Lấy danh mục từ 2 đến 26

    # print("Danh sách các liên kết danh mục từ 2 đến 26:")
    # for idx, href in enumerate(category_links, start=2):
    #     print(f"{idx}: {href}")

    # Thu thập liên kết sản phẩm từ các danh mục
    crawl_category_links(driver, category_links, max_links_per_category=5000)

    print("\nHoàn thành thu thập liên kết sản phẩm.")

    # Đóng trình duyệt
    driver.quit()

if __name__ == "__main__":
    main()



Đang truy cập danh mục 1: https://tiki.vn/nha-cua-doi-song/c1883
Nút 'Xem thêm' không còn hoặc đã tải hết sản phẩm.
Danh mục 1 hoàn thành. Thu thập được 0 liên kết.
Liên kết danh mục 1 đã được lưu vào D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files\category_1.xlsx.

Đang truy cập danh mục 2: https://tiki.vn/dien-thoai-may-tinh-bang/c1789
Collected 1/5000
Collected 2/5000
Collected 3/5000
Collected 4/5000
Collected 5/5000
Collected 6/5000
Collected 7/5000
Collected 8/5000
Collected 9/5000
Collected 10/5000
Collected 11/5000
Collected 12/5000
Collected 13/5000
Collected 14/5000
Collected 15/5000
Collected 16/5000
Collected 17/5000
Collected 18/5000
Collected 19/5000
Collected 20/5000
Collected 21/5000
Collected 22/5000
Collected 23/5000
Collected 24/5000
Collected 25/5000
Collected 26/5000
Collected 27/5000
Collected 28/5000
Collected 29/5000
Collected 30/5000
Collected 31/5000
Collected 32/5000
Collected 33/5000
Collected 3